Prof joshi wanted me to see how closely the features are correlated

In [ ]:
import re
import pandas as pd
pdf = pd.read_stata("IAIR7EFL.DTA",convert_categoricals = False)

In [ ]:
table = pd.read_csv('Table on 27 April.csv')

In [ ]:
table['normal waist circumference'] = 0

In [ ]:
table['normal waist circumference'] += ((pdf['s305'] < 80) & (pdf['s305'] > 50)).astype(int)

In [ ]:
table['normal waist and bmi'] = 0

In [ ]:
table['normal waist and bmi'] += ((table['normal bmi'] == 1) & (table['normal waist circumference'] == 1)).astype(int)

In [ ]:
table

In [ ]:
table['abnormal waist circumference'] = 0
table['abnormal waist or bmi'] = 0
table['abnormal bp'] = 0
table['abnormal glucose'] = 0
table['abnormal haemoglobin'] = 0
table['abnormal bmi'] = 0

In [ ]:
table['abnormal bmi'] = 1 - table['normal bmi']

In [ ]:
table['abnormal bp'] = 1 - table['normal bp']

In [ ]:
table['abnormal haemoglobin'] = 1 - table['Normal haemoglobin']

In [ ]:
table['abnormal waist or bmi'] = 1 - table['normal waist and bmi']

In [ ]:
table['abnormal glucose'] = 1 - table['normal glucose']

In [ ]:
table['abnormal waist circumference'] = 1 - table['normal waist circumference']

In [ ]:
table

In [ ]:
corr1 = table.iloc[:,1:6].corr()

In [ ]:
tab = table.iloc[:,7:]
tab.drop('abnormal waist or bmi', axis = 1, inplace = True)
corr2 = tab.iloc[:,:].corr()

In [ ]:
def heatmap(corr):
    import seaborn as sns
    import matplotlib.pyplot as plt
    plt.figure(figsize=(8, 6))
    sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f", square=True, linewidths=0.5)
    plt.title('Correlation Matrix Heatmap')
    plt.show()


In [ ]:
heatmap(corr1)

In [ ]:
heatmap(corr2)

In [ ]:
table[table['abnormal waist or bmi'] == 1]

In [ ]:
table[table['abnormal bmi'] == 1]

In [ ]:
table.keys()

In [ ]:
from upsetplot import UpSet, from_indicators
import matplotlib.pyplot as plt

# Ensure columns are boolean
table['abnormal waist circumference'] = table['abnormal waist circumference'].astype(bool)
table['abnormal bmi'] = table['abnormal bmi'].astype(bool)

# Create UpSet data
upset_data = from_indicators(['abnormal waist circumference', 'abnormal bmi'], table)

# Increase figure size (e.g., 10 inches wide and 6 inches tall)
plt.figure(figsize=(25, 25))

# Plot
upset = UpSet(upset_data, show_counts=True)
upset.plot()

plt.suptitle("UpSet Plot of Abnormal Waist Circumference and BMI")
plt.show()


In the next few sections, i was trying to cluster the individuals badsed on the metrics: 'abnormal waist circumference', 'abnormal bp', 'abnormal glucose', 'abnormal haemoglobin' and 'abnormal bmi'. I took the help of chatgpt here as my code was taking forever to run.

I realised i was not doing the right thing cause the goal right now was to cluster the featured and not individuals.

So, in the last section (hierarchical clusering), i clustered the features and not individuals.

I have also written down how each alogirthm works for our understanding.

MiniBatchKMeans

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import MiniBatchKMeans

# Step 1: Select features and drop missing values
features = [
    'abnormal waist circumference', 'abnormal bp', 'abnormal glucose',
    'abnormal haemoglobin', 'abnormal bmi'
]

df = table[features].dropna()

# Step 2: Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df)

# Step 3: Run MiniBatchKMeans
k = 3  # You can experiment with other values too
kmeans = MiniBatchKMeans(
    n_clusters=k,
    batch_size=10200000,        # Adjust based on RAM; 10k is a safe starting point
    n_init='auto',           # Faster, modern initialization (>= scikit-learn 1.2)
    random_state=42
)
cluster_labels = kmeans.fit_predict(X_scaled)

# Step 4: Add cluster labels to DataFrame
df['Cluster'] = cluster_labels

# Optional: Save back to original `table`
table.loc[df.index, 'Cluster'] = df['Cluster']

# Step 5: Quick cluster summary
print(df['Cluster'].value_counts())


Visualising cluster (PCA)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming 'df' has scaled health features + 'Cluster' from MiniBatchKMeans
features = [
    'abnormal waist circumference', 'abnormal bp', 'abnormal glucose',
    'abnormal haemoglobin', 'abnormal bmi'
]

# Step 1: Compute cluster-wise means
cluster_summary = df.groupby('Cluster')[features].mean()

# Step 2: Heatmap of cluster-wise feature averages
plt.figure(figsize=(8, 4))
sns.heatmap(cluster_summary, annot=True, fmt=".2f", cmap='YlGnBu', cbar_kws={'label': 'Mean Value'})
plt.title("Cluster-wise Feature Averages")
plt.xlabel("Health Features")
plt.ylabel("Cluster")
plt.tight_layout()
plt.show()

# Step 3: Boxplots per feature by cluster
for feature in features:
    plt.figure(figsize=(6, 4))
    sns.boxplot(data=df, x='Cluster', y=feature, palette='Set2')
    plt.title(f'{feature} by Cluster')
    plt.xlabel("Cluster")
    plt.ylabel(feature)
    plt.tight_layout()
    plt.show()


Using Bernoulli Mixture Model

In [ ]:
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from scipy.stats import bernoulli
from scipy.special import logsumexp
import seaborn as sns

# Assuming your data X is already loaded
N, D = X.shape
K = 5  # number of clusters

print(f"Data shape: {X.shape}")
print(f"Data type: {type(X)}")
print(f"Data range: {X.min().min() if hasattr(X, 'min') else X.min()} to {X.max().max() if hasattr(X, 'max') else X.max()}")

# Convert to numpy if it's a DataFrame
if hasattr(X, 'values'):
    X_np = X.values
else:
    X_np = X

# METHOD 1: Use sklearn's GaussianMixture (works surprisingly well for binary data)
print("\n" + "="*50)
print("METHOD 1: Sklearn GaussianMixture (surprisingly effective for binary data)")

gmm = GaussianMixture(n_components=K, random_state=42, max_iter=100)
gmm.fit(X_np)

# Get predictions
gmm_labels = gmm.predict(X_np)
gmm_probs = gmm.predict_proba(X_np)

print(f"GMM Cluster sizes: {np.bincount(gmm_labels)}")
print(f"GMM Log-likelihood: {gmm.score(X_np):.2f}")
print(f"GMM Converged: {gmm.converged_}")

# METHOD 2: Custom EM Algorithm for Bernoulli Mixture
print("\n" + "="*50)
print("METHOD 2: Custom EM Algorithm for Bernoulli Mixture")

class BernoulliMixture:
    def __init__(self, n_components=2, max_iter=100, tol=1e-6, random_state=None):
        self.n_components = n_components
        self.max_iter = max_iter
        self.tol = tol
        self.random_state = random_state
        
    def fit(self, X):
        if self.random_state:
            np.random.seed(self.random_state)
            
        N, D = X.shape
        K = self.n_components
        
        # Initialize parameters
        self.weights_ = np.ones(K) / K
        self.params_ = np.random.beta(1, 1, size=(K, D))  # Bernoulli parameters
        
        # Initialize responsibilities
        resp = np.random.dirichlet(np.ones(K), N)
        
        log_likelihood_old = -np.inf
        
        for iteration in range(self.max_iter):
            # E-step: compute responsibilities
            log_resp = np.zeros((N, K))
            
            for k in range(K):
                # Log likelihood for cluster k
                log_like_k = np.sum(X * np.log(self.params_[k] + 1e-10) + 
                                  (1 - X) * np.log(1 - self.params_[k] + 1e-10), axis=1)
                log_resp[:, k] = np.log(self.weights_[k] + 1e-10) + log_like_k
            
            # Normalize responsibilities
            log_resp_norm = logsumexp(log_resp, axis=1, keepdims=True)
            log_resp -= log_resp_norm
            resp = np.exp(log_resp)
            
            # M-step: update parameters
            resp_sum = resp.sum(axis=0)
            self.weights_ = resp_sum / N
            
            for k in range(K):
                self.params_[k] = (resp[:, k:k+1].T @ X) / (resp_sum[k] + 1e-10)
                # Clip to avoid numerical issues
                self.params_[k] = np.clip(self.params_[k], 1e-10, 1 - 1e-10)
            
            # Check convergence
            log_likelihood = np.sum(log_resp_norm)
            if abs(log_likelihood - log_likelihood_old) < self.tol:
                print(f"Converged after {iteration + 1} iterations")
                break
            log_likelihood_old = log_likelihood
        
        self.log_likelihood_ = log_likelihood
        return self
    
    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)
    
    def predict_proba(self, X):
        N, D = X.shape
        K = self.n_components
        log_resp = np.zeros((N, K))
        
        for k in range(K):
            log_like_k = np.sum(X * np.log(self.params_[k] + 1e-10) + 
                              (1 - X) * np.log(1 - self.params_[k] + 1e-10), axis=1)
            log_resp[:, k] = np.log(self.weights_[k] + 1e-10) + log_like_k
        
        log_resp_norm = logsumexp(log_resp, axis=1, keepdims=True)
        log_resp -= log_resp_norm
        return np.exp(log_resp)

# Fit the custom Bernoulli mixture
bmm = BernoulliMixture(n_components=K, random_state=42)
bmm.fit(X_np)

bmm_labels = bmm.predict(X_np)
bmm_probs = bmm.predict_proba(X_np)

print(f"BMM Cluster sizes: {np.bincount(bmm_labels)}")
print(f"BMM Log-likelihood: {bmm.log_likelihood_:.2f}")
print(f"BMM Weights: {bmm.weights_}")
print(f"BMM Parameters shape: {bmm.params_.shape}")

# METHOD 3: PyMC for Bayesian approach (if you have PyMC installed)
print("\n" + "="*50)
print("METHOD 3: PyMC Bayesian Bernoulli Mixture (requires PyMC)")

try:
    import pymc as pm
    import pytensor.tensor as pt
    
    # Sample a subset for faster computation
    sample_size = min(10000, N)
    idx = np.random.choice(N, sample_size, replace=False)
    X_sample = X_np[idx]
    
    with pm.Model() as model:
        # Priors
        weights = pm.Dirichlet('weights', a=np.ones(K))
        params = pm.Beta('params', alpha=1, beta=1, shape=(K, D))
        
        # Cluster assignment
        cluster = pm.Categorical('cluster', p=weights, shape=sample_size)
        
        # Likelihood
        obs = pm.Bernoulli('obs', p=params[cluster], observed=X_sample)
        
        # Inference
        trace = pm.sample(1000, tune=500, chains=2, random_seed=42)
    
    # Get posterior means
    pymc_weights = trace.posterior['weights'].mean(dim=['chain', 'draw']).values
    pymc_params = trace.posterior['params'].mean(dim=['chain', 'draw']).values
    
    print(f"PyMC Weights: {pymc_weights}")
    print(f"PyMC Parameters shape: {pymc_params.shape}")
    
except ImportError:
    print("PyMC not installed - skipping Bayesian method")
except Exception as e:
    print(f"PyMC method failed: {e}")

# METHOD 4: Simple K-means clustering (baseline)
print("\n" + "="*50)
print("METHOD 4: K-means baseline")

kmeans = KMeans(n_clusters=K, random_state=42, n_init=10)
kmeans_labels = kmeans.fit_predict(X_np)

print(f"K-means Cluster sizes: {np.bincount(kmeans_labels)}")

# Compare results
print("\n" + "="*50)
print("COMPARISON OF METHODS")

methods = {
    'GMM': gmm_labels,
    'Bernoulli MM': bmm_labels,
    'K-means': kmeans_labels
}

for name, labels in methods.items():
    print(f"\n{name}:")
    print(f"  Cluster sizes: {np.bincount(labels)}")
    if hasattr(X, 'columns'):
        # If X is a DataFrame, show feature means by cluster
        cluster_means = pd.DataFrame(X_np).groupby(labels).mean()
        print(f"  Feature means by cluster:")
        print(cluster_means.round(3))

# Visualization if reasonable size
if D <= 10:
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    axes = axes.ravel()
    
    for i, (name, labels) in enumerate(methods.items()):
        if i < len(axes):
            # Plot first two features colored by cluster
            scatter = axes[i].scatter(X_np[:, 0], X_np[:, 1], c=labels, cmap='viridis', alpha=0.6)
            axes[i].set_title(f'{name} Clustering')
            axes[i].set_xlabel('Feature 0')
            axes[i].set_ylabel('Feature 1')
            plt.colorbar(scatter, ax=axes[i])
    
    plt.tight_layout()
    plt.show()

print("\nRecommendation:")
print("1. For simplicity and speed: Use sklearn's GaussianMixture")
print("2. For proper Bernoulli modeling: Use the custom BernoulliMixture class")
print("3. For Bayesian inference: Use PyMC (if installed)")
print("4. For baseline comparison: K-means")

# Mathematical Explanation of Clustering Methods for Binary Data

---

## 1. Bernoulli Mixture Model (BMM)

The Bernoulli Mixture Model assumes each data point \( \mathbf{x}_n = (x_{n1}, x_{n2}, \dots, x_{nD}) \) is generated by one of \( K \) clusters.

Each cluster \( k \) has:

- A **Bernoulli probability vector**:  
  $$ \mathbf{p}_k = (p_{k1}, p_{k2}, \dots, p_{kD}) $$

- A **cluster prior** (mixture weight):  
  $$ \pi_k, \quad \text{where} \quad \sum_{k=1}^K \pi_k = 1 $$


### Likelihood for a single data point:

$$
P(\mathbf{x}_n \mid z_n = k) = \prod_{d=1}^D p_{kd}^{x_{nd}} (1 - p_{kd})^{1 - x_{nd}}
$$

---

### EM Algorithm

#### **E-step:** Compute responsibilities (posterior cluster probabilities)

$$
\gamma_{nk} = \frac{
\pi_k \prod_{d=1}^D p_{kd}^{x_{nd}} (1 - p_{kd})^{1 - x_{nd}}
}{
\sum_{j=1}^K \pi_j \prod_{d=1}^D p_{jd}^{x_{nd}} (1 - p_{jd})^{1 - x_{nd}}
}
$$

#### **M-step:** Update parameters

Update mixture weights:

$$
\pi_k = \frac{1}{N} \sum_{n=1}^N \gamma_{nk}
$$

Update Bernoulli parameters:

$$
p_{kd} = \frac{\sum_{n=1}^N \gamma_{nk} x_{nd}}{\sum_{n=1}^N \gamma_{nk}}
$$

---

## 2. Gaussian Mixture Model (GMM)

The Gaussian Mixture Model assumes each data point is generated from a Gaussian distribution:

$$
p(\mathbf{x}_n) = \sum_{k=1}^K \pi_k \mathcal{N}(\mathbf{x}_n \mid \boldsymbol{\mu}_k, \Sigma_k)
$$

- \( \boldsymbol{\mu}_k \): mean of cluster \( k \)
- \( \Sigma_k \): covariance matrix of cluster \( k \)
- \( \mathcal{N}(\cdot) \): multivariate Gaussian PDF

Although GMMs are designed for continuous data, they often work reasonably for binary data due to flexibility in modeling.

---

## 3. K-means Clustering

K-means partitions data into \( K \) clusters by minimizing intra-cluster variance:

$$
\arg\min_{C} \sum_{k=1}^K \sum_{\mathbf{x}_n \in C_k} \|\mathbf{x}_n - \boldsymbol{\mu}_k\|^2
$$

- \( \boldsymbol{\mu}_k \): centroid of cluster \( k \)

K-means is simple and fast but lacks probabilistic interpretation and is less suitable for binary data.

---

## Summary

| Method   | Suitable for Binary? | Probabilistic? | Notes |
|----------|----------------------|----------------|-------|
| BMM      | ✅ Yes               | ✅ Yes         | Tailored for binary features |
| GMM      | ⚠️ Acceptable        | ✅ Yes         | Assumes continuous features |
| K-means  | ⚠️ Weak              | ❌ No          | Fast baseline, but not ideal for binary data |



Hierarchical clustering


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from sklearn.metrics import jaccard_score
import seaborn as sns

feature_names = [
    'abnormal waist circumference', 'abnormal bp', 'abnormal glucose',
    'abnormal haemoglobin', 'abnormal bmi'
]

np.random.seed(42)
X = table[[
    'abnormal waist circumference', 'abnormal bp', 'abnormal glucose',
    'abnormal haemoglobin', 'abnormal bmi'
]]


D = X.shape[1]

#Jaccard similarity matrix
jaccard_similarity = np.zeros((D, D))
for i in range(D):
    for j in range(D):
        jaccard_similarity[i, j] = jaccard_score(X.iloc[:, i], X.iloc[:, j])


jaccard_distance = 1 - jaccard_similarity

# Perform hierarchical clustering using average linkage on the distance matrix
linkage_matrix = linkage(jaccard_distance, method='average')

# no. of clusters
num_clusters = 3
num_clusters = min(num_clusters, D)  # safety

# cluster assignments
clusters = fcluster(linkage_matrix, num_clusters, criterion='maxclust')

# creating labels with feature name and cluster number for dendrogram leaves
labels_with_cluster = [f"{name} (C{clusters[i]})" for i, name in enumerate(feature_names)]

# dendrogram
plt.figure(figsize=(16, 7))
dendro = dendrogram(
    linkage_matrix,
    labels=labels_with_cluster,
    leaf_rotation=45,
    leaf_font_size=12,
    color_threshold=0
)

#horizontal line for cluster threshold
threshold_idx = linkage_matrix.shape[0] - num_clusters
threshold = linkage_matrix[threshold_idx, 2]
plt.axhline(y=threshold, color='grey', linestyle='--', linewidth=1.5, label='Cluster Threshold')

plt.title('Hierarchical Clustering Dendrogram of Binary Features\n(with Jaccard Distance)')
plt.xlabel('Features')
plt.ylabel('Distance')
plt.legend()
plt.tight_layout()
plt.show()

# Heatmap of jaccard similarity
plt.figure(figsize=(10, 8))
sns.heatmap(
    jaccard_similarity,
    xticklabels=feature_names,
    yticklabels=feature_names,
    cmap='viridis',
    annot=True,
    fmt=".2f",
    square=True,
    cbar_kws={'label': 'Jaccard Similarity'}
)
plt.title('Feature Similarity Heatmap (Jaccard Index)')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

print("Cluster assignments and features:")
for c in range(1, num_clusters + 1):
    members = [feature_names[i] for i in range(D) if clusters[i] == c]
    print(f"Cluster {c} ({len(members)} features): {', '.join(members)}")


# Explanation of Metrics, Methods, and Parameters Used in the Binary Feature Clustering Code

## 1. Jaccard Similarity / Distance

- **Jaccard Similarity** measures how similar two binary vectors are:

  $$
  J(A, B) = \frac{|A \cap B|}{|A \cup B|}
  $$

  - Where \( A \) and \( B \) are sets of positions where the binary features are 1.
  - The numerator counts positions where both vectors have 1.
  - The denominator counts positions where either vector has 1.

- **Range**:
  - \( J = 1 \): features are identical.
  - \( J = 0 \): features have no overlap (dissimilar).

- **Jaccard Distance** is:

  $$
  D(A, B) = 1 - J(A, B)
  $$

- In code, we use `sklearn.metrics.jaccard_score` (with `average=None`) to compute similarity across feature pairs.

---

## 2. Hierarchical (Agglomerative) Clustering

- Start with each feature as its own cluster.
- At each step, merge the two closest clusters based on a distance measure.

- With **average linkage**, the distance between clusters \( C_i \) and \( C_j \) is:

  $$
  d(C_i, C_j) = \frac{1}{|C_i| \cdot |C_j|} \sum_{x \in C_i} \sum_{y \in C_j} d(x, y)
  $$

  - Where \( d(x, y) \) is the Jaccard distance between feature vectors \( x \) and \( y \).
  - Average linkage computes the mean of all pairwise distances between members of the two clusters.

